## 1. 데이터 불러오기

In [ ]:
import pandas as pd

# 학습 데이터 로딩
df17 = pd.read_csv('./data/data2017.csv')
df18 = pd.read_csv('./data/data2018.csv')
df19 = pd.read_csv('./data/data2019.csv')
df20 = pd.read_csv('./data/data2020.csv')
df21 = pd.read_csv('./data/data2021.csv')
df22 = pd.read_csv('./data/data2022.csv')

In [ ]:
df17.head()

,기준일ID,시간대구분,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20170101,0,31535.2200,1945.4402,566.4588,631.5581,991.9184,1297.1287,1223.1529,1552.9448,...,1157.6661,1107.6781,1282.9235,1029.4360,998.0414,874.5643,1162.1547,1053.0477,964.3940,2025.4819
1,20170101,1,31188.9174,1930.4766,562.1018,625.2151,1030.6281,1307.7381,1183.0956,1491.5827,...,1120.0051,1102.5198,1271.8997,1027.2574,1038.7528,857.9100,1149.7532,1042.7411,919.0115,2062.2422
2,20170101,2,31240.4974,1921.1864,559.3965,658.7463,1058.7082,1311.9591,1186.2600,1537.3732,...,1103.1547,1112.4579,1282.9366,1027.8901,1020.0177,853.3890,1193.4472,1011.2073,889.9325,1960.1236
3,20170101,3,31442.4314,1946.4476,566.7525,649.0804,1057.9119,1324.2638,1202.1250,1521.2732,...,1116.2165,1102.0668,1260.8472,1007.6650,1042.9554,896.0038,1163.0355,1056.6428,891.9177,2060.1872
4,20170101,4,31922.7751,1953.7713,568.8845,623.2087,1063.2557,1342.9146,1209.2680,1556.0675,...,1187.5457,1120.2232,1281.2738,1064.0655,1045.2130,895.8336,1191.7017,1045.3362,874.4984,2038.6212


In [ ]:
df_total = pd.concat([df17, df18, df19, df20, df21], axis=0)

---

## 2. 데이터 전처리

In [ ]:
# 열 추가
df_total['기준일ID'] = pd.to_datetime(df_total['기준일ID'], format='%Y%m%d')

df_total['year']=df_total['기준일ID'].dt.year
df_total['month']=df_total['기준일ID'].dt.month
df_total['day']=df_total['기준일ID'].dt.day

In [ ]:
# 열 제거 및 이름 변경
df_total.rename(columns={'시간대구분': 'hour'}, inplace=True)
df_total = df_total.loc[:, ['year', 'month', 'day', 'hour', '총생활인구수']].copy()

In [ ]:
# 시프트를 사용해 target 설정
df_total['총생활인구수_M2'] = df_total['총생활인구수'].shift(-1416) # 2개월 후 동일 시간대의 총 생활인구수

In [ ]:
# 롤링을 사용해 feature 추가
df_total['총생활인구수_MA_1W'] = df_total['총생활인구수'].rolling(168).mean() # 기준일 포함 최근 7일간 이동 평균
df_total['총생활인구수_MA_2W'] = df_total['총생활인구수'].rolling(336).mean() # 기준일 포함 최근 14일간 이동 평균
df_total['총생활인구수_MA_1M'] = df_total['총생활인구수'].rolling(720).mean() # 기준일 포함 최근 30일간 이동 평균
df_total['총생활인구수_MA_2M'] = df_total['총생활인구수'].rolling(1440).mean() # 기준일 포함 최근 60일간 이동 평균

In [ ]:
df_total.tail()

,year,month,day,hour,총생활인구수,총생활인구수_M2,총생활인구수_MA_1W,총생활인구수_MA_2W,총생활인구수_MA_1M,총생활인구수_MA_2M
8755,2021,12,31,19,33060.4555,NaN,35189.981485,35229.873104,35268.758925,35429.657340
8756,2021,12,31,20,32360.1898,NaN,35185.892541,35227.022501,35268.342115,35428.862936
8757,2021,12,31,21,31417.3991,NaN,35182.222624,35226.172293,35267.912928,35428.193338
8758,2021,12,31,22,30763.0563,NaN,35176.888761,35221.590352,35266.813773,35427.355402
8759,2021,12,31,23,30404.2740,NaN,35166.708061,35218.038761,35264.280282,35426.113432


In [ ]:
datasets = []

In [ ]:
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1W', '총생활인구수_M2']].copy())
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_2W', '총생활인구수_M2']].copy())
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1M', '총생활인구수_M2']].copy())

In [ ]:
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1W', '총생활인구수_MA_2W', '총생활인구수_M2']].copy())
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1W', '총생활인구수_MA_1M', '총생활인구수_M2']].copy())
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1W', '총생활인구수_MA_2M', '총생활인구수_M2']].copy())
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1M', '총생활인구수_MA_2M', '총생활인구수_M2']].copy())

In [ ]:
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1W', '총생활인구수_MA_2W', '총생활인구수_MA_1M', '총생활인구수_M2']].copy())
datasets.append(df_total.loc[:, ['총생활인구수', '총생활인구수_MA_1W', '총생활인구수_MA_2W', '총생활인구수_MA_1M', '총생활인구수_MA_2M', '총생활인구수_M2']].copy())

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
for dataset in datasets:
    dataset.dropna(axis=0, inplace=True)

In [ ]:
explainer = [
    '--- 7일간 이동 평균 feature 추가 ---',
    '--- 14일간 이동 평균 feature 추가 ---',
    '--- 30일간 이동 평균 feature 추가 ---',
    '--- 7일 + 14일간 이동 평균 feature 추가 ---',
    '--- 7일 + 30일간 이동 평균 feature 추가 ---',
    '--- 7일 + 60일간 이동 평균 feature 추가 ---',
    '--- 30일 + 60일간 이동 평균 feature 추가 ---',
    '--- 7일 + 14일 + 30일간 이동 평균 feature 추가 ---',
    '--- 7일 + 14일 + 30일 + 60일간 이동 평균 feature 추가 ---'
]

### train, test 데이터 나누기

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = [[None] for _ in range(10)], [[None] for _ in range(10)], [[None] for _ in range(10)], [[None] for _ in range(10)]

In [ ]:
for i, dataset in enumerate(datasets):
    x = dataset.drop('총생활인구수_M2', axis=1)
    y = dataset['총생활인구수_M2'].values

    train_x[i], test_x[i], train_y[i], test_y[i] = train_test_split(x, y, test_size=.2)

----

## 3. 모델링

In [ ]:
from sklearn.metrics import *

### 가. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

for i, (tr_x, tr_y, tx, ty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    rf = RandomForestRegressor()
    rf.fit(tr_x, tr_y)
    
    pred_rf = rf.predict(tx)

    print(explainer[i])
    print('RMSE: ', mean_squared_error(ty, pred_rf, squared=False))
    print('R-squared Score: ', r2_score(ty, pred_rf))
    print()

--- 7일간 이동 평균 feature 추가 ---
RMSE:  2897.83487922053
R-squared Score:  0.5526510881960528

--- 14일간 이동 평균 feature 추가 ---
RMSE:  2815.023019711814
R-squared Score:  0.5674859787508981

--- 30일간 이동 평균 feature 추가 ---
RMSE:  2780.6566387505336
R-squared Score:  0.5835547368840184

--- 7일 + 14일간 이동 평균 feature 추가 ---
RMSE:  2053.6228344729893
R-squared Score:  0.7757295855351811



### 나. XGB

In [ ]:
# 아래에 실습코드를 작성하세요.
from xgboost import XGBRegressor

for i, (tr_x, tr_y, tx, ty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    xgb = XGBRegressor()
    xgb.fit(tr_x, tr_y.ravel())
    
    pred_xgb = xgb.predict(tx)

    print(explainer[i])        
    print('RMSE: ', mean_squared_error(ty, pred_xgb, squared=False))
    print('R-squared Score: ', r2_score(ty, pred_xgb))
    print()

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

### 다. LightGBM

In [ ]:
import lightgbm as lgb

for i, (tr_x, tr_y, tx, ty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    lgbm = lgb.LGBMRegressor()
    lgbm.fit(tr_x, tr_y.ravel())
    
    pred_lgbm = lgbm.predict(tx)

    print(explainer[i])        
    print('RMSE: ', mean_squared_error(ty, pred_lgbm, squared=False))
    print('R-squared Score: ', r2_score(ty, pred_lgbm))
    print()

LGBMRegressor()

### 라. CatBoost

In [ ]:
import catboost

for i, (tr_x, tr_y, tx, ty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    cb = catboost.CatBoostRegressor()
    cb.fit(tr_x, tr_y.ravel())
    
    pred_cb = cb.predict(tx)

    print(explainer[i])        
    print('RMSE: ', mean_squared_error(ty, pred_cb, squared=False))
    print('R-squared Score: ', r2_score(ty, pred_cb))
    print()